In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json

In [2]:
# Check if MPS is available
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Check if CUDA is available
# device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-coder-1.3b-instruct",
    trust_remote_code=True,
    torch_dtype=torch.float16  # MPS prefers float16 instead of bfloat16
).to(device)

# If you are using CUDA, you can use bfloat16 instead of float16
# model = AutoModelForCausalLM.from_pretrained(
#     "deepseek-ai/deepseek-coder-1.3b-instruct",
#     trust_remote_code=True,
#     torch_dtype=torch.bfloat16
# ).to(device)

print("Running on:", device)

Running on: mps


In [4]:
def generate_output(messages):
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    # tokenizer.eos_token_id is the id of <|EOT|> token
    outputs = model.generate(inputs, max_new_tokens=2048, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
    description = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    return description

In [5]:
def string_to_json(input_string):
    try:
        data = json.loads(input_string)
        return data, None  # Return parsed data and no error
    except json.JSONDecodeError as e:
        return None, str(e) # Return None and the error message

In [18]:
def dictionary_to_json(data_dict, filepath, indent=4):
    try:
        with open(filepath, 'r') as f:
            try:
                existing_data = json.load(f)
                if isinstance(existing_data, list):
                    existing_data.append(data_dict)
                else:
                    existing_data = [existing_data, data_dict] # if the first insertion was not a list, turn it into a list.
            except json.JSONDecodeError:
                existing_data = [data_dict] # handles empty file
    except FileNotFoundError:
        existing_data = [data_dict]  # File doesn't exist, create it

    with open(filepath, 'w') as f:
        json.dump(existing_data, f, indent=4) # indent makes the json file human readable

In [7]:
def read_code_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

In [8]:
file_path = "Code-Files/FactorialRecursive.java"
code_snippet = read_code_from_file(file_path)

In [9]:
prompt = """
You are an expert code analyst. Your task is to dissect a given code snippet and provide a structured JSON response detailing its components and overall functionality. You must strictly adhere to the following JSON format:
{
  "output": {
    "components": [
      {
        "component_type": "<TYPE_OF_CODE_COMPONENT (e.g., CLASS, FUNCTION, GLOBAL_CODE)>",
        "component_name": "<NAME_OF_COMPONENT>",
        "component_code": "<THE_ACTUAL_CODE_OF_THE_COMPONENT>",
        "component_description": "<DETAILED_DESCRIPTION_OF_COMPONENT_FUNCTIONALITY>"
      },
      { /* ... more components ... */ }
    ],
    "overall_description": "<SUMMARY_OF_THE_ENTIRE_CODE_FUNCTIONALITY>"
  }
}

**Instructions:**
1.  **Strict JSON Output:** Your entire response must be a valid JSON object. No extraneous text or explanations outside the JSON structure are allowed.
2.  **Component Types:** Accurately identify the type of each component (e.g., CLASS, FUNCTION, GLOBAL_CODE, etc.).
3.  **Component Names:** Provide the correct name of each component (e.g., class name, function name, etc.).
4.  **Component Code:** provide the actual code of the component.
5.  **Component Descriptions:** For each component, provide a detailed description of its purpose and functionality.
6.  **Overall Description:** Provide a summary of the entire code snippet, explaining its overall purpose and how the components work together.

**Example:**
{
  "output": {
    "components": [
      {
        "component_type": "CLASS",
        "component_name": "Person",
        "component_code": "class Person:\n    def __init__(self, name, age):\n        self.name = name\n        self.age = age\n\n    def greet(self):\n        print(f'Hello, my name is {self.name} and I am {self.age} years old.)",
        "component_description": "Defines the Person class with attributes for name and age and a method for greeting."
      },
      {
        "component_type": "FUNCTION",
        "component_name": "__init__",
        "component_code": "def __init__(self, name, age):\n        self.name = name\n        self.age = age",
        "component_description": "Constructor to initialize a Person object with name and age."
      },
      {
        "component_type": "FUNCTION",
        "component_name": "greet",
        "component_code": "def greet(self):\n        print(f'Hello, my name is {self.name} and I am {self.age} years old.')",
        "component_description": "Greets the user with the name and age stored in the Person object."
      },
      {
        "component_type": "GLOBAL_CODE",
        "component_name": "__main__ block",
        "component_code": "if __name__ == '__main__':\n    p = Person('John', 30)\n    p.greet()",
        "component_description": "Creates a Person object and calls the greet method to display the greeting."
      }
    ],
    "overall_description": "This Python code defines a class called Person with methods for initializing the object and greeting the user. It demonstrates object-oriented programming by creating a Person object and calling its methods."
  }
}

Now analyze the following code snippet and provide the JSON response:\n
"""

In [10]:
messages=[
    { 'role': 'user', 'content': prompt+code_snippet}
]

In [11]:
output = generate_output(messages)

/opt/miniconda3/envs/malcodeai/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [12]:
print(output)

{
  "output": {
    "components": [
      {
        "component_type": "CLASS",
        "component_name": "FactorialRecursive",
        "component_code": "public class FactorialRecursive {\n\n    public static int factorial(int n) {\n        // Base case: factorial of 0 is 1\n        if (n == 0) {\n            return 1;\n        } else {\n            // Recursive step: n! = n * (n-1)!\n            return n * factorial(n - 1);\n        }\n    }\n\n    public static void main(String[] args) {\n        int num = 5;  // Example: Calculate factorial of 5\n        int result = factorial(num);\n        System.out.println(\"Factorial of \" + num + \" is: \" + result);\n\n        //Demonstrating potential issues with large numbers\n        num = 20;\n        result = factorial(num);\n        System.out.println(\"Factorial of \" + num + \" is: \" + result);\n\n        num = 25; //Factorial of 25 is a very large number that will exceed int's maximum.\n        result = factorial(num);\n        Syst

In [13]:
parsed_data, error = string_to_json(output)

if parsed_data:
    print("Valid JSON:")
    print(parsed_data)
else:
    print("Invalid JSON:", error)

Valid JSON:
{'output': {'components': [{'component_type': 'CLASS', 'component_name': 'FactorialRecursive', 'component_code': 'public class FactorialRecursive {\n\n    public static int factorial(int n) {\n        // Base case: factorial of 0 is 1\n        if (n == 0) {\n            return 1;\n        } else {\n            // Recursive step: n! = n * (n-1)!\n            return n * factorial(n - 1);\n        }\n    }\n\n    public static void main(String[] args) {\n        int num = 5;  // Example: Calculate factorial of 5\n        int result = factorial(num);\n        System.out.println("Factorial of " + num + " is: " + result);\n\n        //Demonstrating potential issues with large numbers\n        num = 20;\n        result = factorial(num);\n        System.out.println("Factorial of " + num + " is: " + result);\n\n        num = 25; //Factorial of 25 is a very large number that will exceed int\'s maximum.\n        result = factorial(num);\n        System.out.println("Factorial of " + nu

In [14]:
parsed_data['input'] = code_snippet

In [16]:
parsed_data.keys()

dict_keys(['output', 'input'])

In [20]:
sorted_keys = sorted(parsed_data.keys())
final_data = {key: parsed_data[key] for key in sorted_keys}

In [21]:
dictionary_to_json(final_data, "Sample-JSON.json", indent=4)